In [6]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate


   ###

    
#  How to use tabulate-
#
l_result = [{ "col1": 20, "col2": 30}]
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))


print("--")


+--------+--------+
|   col1 |   col2 |
|--------+--------|
|     20 |     30 |
+--------+--------+
--


In [1]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [2]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [3]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 5sence62JumWdFFiRo2bUE8yBTTbp3VShtTTK44K4YW7, 1>


In [4]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

          0/? [?op/s]

29946

          0/? [?op/s]

43072

In [ ]:
# ##################################################################

In [ ]:

#  Making a GS Bucket Public,
#
#  From,
#     https://cloud.google.com/storage/docs/access-control/making-data-public#command-line

gsutil iam ch allUsers:objectViewer gs://farrell-bucket


#  In browser,
#
#     https://console.cloud.google.com/storage/browser/farrell-bucket


In [14]:

#  Read CSV from GS

import os
   #
import dask.dataframe as dd
import numpy as np
   #
from google.cloud import storage
import gcsfs


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/work/03_MyKeys.json"


l_bucket    = "katana-demo-datasets"
l_fullpath  = "gs://" + l_bucket + "/csv-datasets/ldbc/sf-0.003/csv/bi/composite-projected-fk/initial_snapshot/static/Place/part-00000-c16729dd-16d5-456b-84ca-d0f19fa783d6-c000.csv"


l_dataframe = dd.read_csv(
   l_fullpath,
   delimiter='|',
   dtype={
      'id':   int,
      "name": np.dtype('O'),
      "url":  np.dtype('O'),
      "type": np.dtype('O'),
      })


l_cntr = 0
   #
for l_each in l_dataframe.itertuples():
   l_cntr+=1
      #
   if (l_cntr < 10):
      print("Id: %-4d    Name: %-28s   Url: %-60s   Type: %s" % (l_each.id, l_each.name, l_each.url, l_each.type))
    
print("")
    

Id: 0       Name: India                          Url: http://dbpedia.org/resource/India                              Type: Country
Id: 1       Name: China                          Url: http://dbpedia.org/resource/China                              Type: Country
Id: 2       Name: Angola                         Url: http://dbpedia.org/resource/Angola                             Type: Country
Id: 3       Name: Austria                        Url: http://dbpedia.org/resource/Austria                            Type: Country
Id: 4       Name: Azerbaijan                     Url: http://dbpedia.org/resource/Azerbaijan                         Type: Country
Id: 5       Name: Bolivia                        Url: http://dbpedia.org/resource/Bolivia                            Type: Country
Id: 6       Name: Bosnia_and_Herzegovina         Url: http://dbpedia.org/resource/Bosnia_and_Herzegovina             Type: Country
Id: 7       Name: Bulgaria                       Url: http://dbpedia.org/resource/B

In [15]:

#  Output the above to GS Parquet
#
#     See,
#        https://cloud.google.com/iam/docs/creating-managing-service-account-keys#iam-service-account-keys-list-gcloud
#
#        gcloud iam service-accounts keys list --iam-account=farrell@katana-internal39.iam.gserviceaccount.com


l_outputfolder = "gs://" + "farrell-bucket" + "/test"


l_dataframe.to_parquet(l_outputfolder,
   write_index         = False,
   write_metadata_file = False,
      #
   engine              = "pyarrow",
   version             = "2.6"
   )

print("--")


--


In [20]:

#  Read the above

df_data = dd.read_parquet("https://storage.googleapis.com/farrell-bucket/test/part.0.parquet")


#  df_data.head(5)

l_cntr = 0
   #
for l_each in df_data.itertuples():
   l_cntr += 1
      #
   if (l_cntr < 5):
      print("Id: %-4d    Name: %-28s   Url: %-60s   Type: %s" % (l_each.id, l_each.name, l_each.url, l_each.type))

print("--")    


Id: 0       Name: India                          Url: http://dbpedia.org/resource/India                              Type: Country
Id: 1       Name: China                          Url: http://dbpedia.org/resource/China                              Type: Country
Id: 2       Name: Angola                         Url: http://dbpedia.org/resource/Angola                             Type: Country
Id: 3       Name: Austria                        Url: http://dbpedia.org/resource/Austria                            Type: Country
--


In [21]:

#  Import GS Parquet data into graph

from katana.remote import import_data

l_node ="gs://farrell-bucket/test/50_NodeHeader.txt"
l_data ="gs://farrell-bucket"

import_data.parquet(
   my_graph,
      #
   input_node_path   = l_node,
   #  input_edge_path   = l_edge,
   input_dir         = l_data,
      #
   schema_delimiter="|",
   ids_are_integers=True
   )

print("--")


          0/? [?op/s]

OperationError: DQqHbBa7RRPpSYsyFiDw9yiGJp98rJTLyWrAGMkJZqh4-R6WMR2585ncCDuQv backtrace (ImportOperation.cpp:182): backtrace (ImportOperation.cpp:154): backtrace (CSVImport.cpp:830): backtrace (ArrowTableGraphBuilder.cpp:2558): backtrace (ArrowTableGraphBuilder.cpp:2595): backtrace (ArrowTableGraphBuilder.cpp:1290): backtrace (DistributedGIDLookupTable.h:95): backtrace (DistributedGIDLookupTable.h:561): backtrace (LogStructuredPropertyGraph.cpp:1120): no column with name ID (PropertyTableGroup.h:161): no such property: no such property